# Test ML Library  
- CPU環境ではTensorflowからGPUが使用されていないことに関する警告が表示される可能性がる．  

In [1]:
import warnings

from ml_utils import *

warnings.filterwarnings('ignore')

2021-11-02 21:56:15.017787: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.9/site-packages/cv2/../../lib64:
2021-11-02 21:56:15.017809: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Load Data  

In [2]:
A_IMGPATH = "data/taskA/img"
A_DFPATH = "data/taskA/table"
B_IMGPATH = "data/taskB/img"
B_DFPATH = "data/taskB/table"
asset_df_A = pd.read_csv(os.path.join(A_DFPATH, "asset_data.csv"))
asset_df_B = pd.read_csv(os.path.join(B_DFPATH, "asset_data.csv"))

asset_df_A = pd.concat((asset_df_A, pd.get_dummies(asset_df_A['asset_contract.name'])), axis=1)
asset_df_B[asset_df_A.columns.values[8:]] = 0
asset_df_B = asset_df_B.rename(columns={"asset.num_sales": "num_sales"})
asset_df_A = asset_df_A.rename(columns={"last_sale.total_price": "current_price"})
asset_df_A['current_price'] = asset_df_A['current_price'].astype(float)

asset_df_A["full_path"] =\
    asset_df_A["image_id"].apply(lambda x: A_IMGPATH + "/" + x)
asset_df_B["full_path"] =\
    asset_df_B["image_id"].apply(lambda x: B_IMGPATH + "/" + x)

print(f"data shape: {asset_df_A.shape}")
print(f"data shape: {asset_df_B.shape}")

data shape: (4146, 18)
data shape: (37306, 23)


### Task A Model  

In [3]:
file_name = "models/efn_baselineA.pkl"
model = load_model(file_name)
pred = model.predict(asset_df_A['full_path'][0], asset_df_A['asset_contract.name'][0],
                     asset_df_A['num_sales'][0])
print(pred)

2021-11-02 21:56:16.497654: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.9/site-packages/cv2/../../lib64:
2021-11-02 21:56:16.497674: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-11-02 21:56:16.497705: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:163] no NVIDIA GPU device is present: /dev/nvidia0 does not exist
2021-11-02 21:56:16.497836: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-02 21:56:18.401591: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185]

0.06589031


### Task B Model

In [4]:
file_name = "models/efn_baselineB.pkl"
model = load_model(file_name)
pred = model.predict(asset_df_B['full_path'][1], None,
                     asset_df_B['num_sales'][1])
print(pred)

0.08978978
